# Library Install

In [ ]:
!pip install -q pydantic-ai pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.4/420.4 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.8/376.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
from google.colab import userdata

from pydantic_ai.providers.google import GoogleProvider
from pydantic_ai.models.google import GoogleModel
from pydantic_ai import Agent

import os
import requests
import fitz

import numpy as np
import google.generativeai as genai

In [4]:
from IPython.display import display, Markdown

def nbprint(s):
    display(Markdown(s))

# Setup Gemini model via PydanticAI


In [5]:
os.environ["GEMINI_MODEL"]  = "gemini-2.5-flash-lite"
os.environ["GEMINI_KEY"] = userdata.get('GeminiKey')

In [6]:
genai.configure(api_key=os.environ["GEMINI_KEY"])

provider = GoogleProvider(api_key=os.environ["GEMINI_KEY"])
model = GoogleModel(os.environ["GEMINI_MODEL"], provider=provider)
rag_agent = Agent(model=model,
                  system_prompt="You are a RAG assistant. Use ONLY the context to answer clearly.")

response = await rag_agent.run("""World ended in 2012 and now it is 2025.
 How many years have passed since World ended?""")
nbprint(response.output)


The world ended in 2012. The current year is 2025.
Therefore, 2025 - 2012 = 13 years have passed since the world ended.

# 📘 Data Store Preparation Pipeline

- 🗎 **Download PDF** → Get the source document  
- 📄 **Extract Text** → Convert PDF into raw text  
- ✂️ **Chunk Text** → Break text into smaller pieces  
- 🔢 **Generate Embeddings** → Represent chunks as vectors  
- 📦 **Prepare Vector Store** → Store embeddings for retrieval  


# Fetch + Parse PDF

In [8]:
url = "https://arxiv.org/pdf/2510.25137.pdf"
pdf_bytes = requests.get(url).content

with open("temp.pdf", "wb") as f:
    f.write(pdf_bytes)

doc = fitz.open("temp.pdf")

full_text = ""
for page in doc:
    full_text += page.get_text("text") + "\n"

nbprint(full_text[:100])


PROJECT ICEBERG
The Iceberg Index:
Measuring Workforce Exposure in the
AI Economy
Ayush Chopra1,6
Sa

# Chunking

In [18]:
def chunk_text(text, max_chars=1000, overlap=5):
    chunks = []
    for i in range(0, len(text), max_chars):
      start = max(0, i-overlap)
      end = min(len(text), i+max_chars+overlap)
      chunks.append(text[start:end])
    return chunks

chunks = chunk_text(full_text, max_chars=1000)
print(len(chunks))
nbprint(chunks[0])

64


PROJECT ICEBERG
The Iceberg Index:
Measuring Workforce Exposure in the
AI Economy
Ayush Chopra1,6
Santanu Bhattacharya1,6
DeAndrea Salvador3,4,1
Ayan Paul6
Teddy
Wright5
Aditi Garg6
Feroz Ahmad6
Alice C. Schwarze5
Ramesh Raskar1,6
Prasanna Balaprakash2
1Massachusetts Institute of Technology
2Oak Ridge National Laboratory
3North Carolina General Assembly
4National Task Force on State AI Policy
5Utah Office of AI Policy, Dept of Commerce
6Project Iceberg
Abstract. Artificial Intelligence is reshaping America’s over $9.4 trillion labor market, with
cascading effects that extend far beyond visible technology sectors. When AI automates
quality control in automotive plants, consequences spread through logistics networks,
supply chains, and local service economies. Yet traditional workforce metrics cannot
capture these ripple effects: they measure employment outcomes after disruption occurs,
not where AI capabilities overlap with human skills before adoption crystallizes. Project
Iceberg addresses

# Embedding and Vector Store

In [19]:
def embed(text):
    resp = genai.embed_content(model="text-embedding-004",
                               content=text,
                               task_type="retrieval_document")
    return np.array(resp["embedding"], dtype=float)

embeddings = np.array([embed(c) for c in chunks])
print(len(embeddings))
print(embeddings[0][:5])


64
[ 0.03459131 -0.00489019 -0.05584043  0.05050938  0.0224702 ]


# 💬 User Query Answering Pipeline

- 🙋 **User Query** → Input from the user  
- 🔎 **Embed Query** → Convert query into vector  
- 📂 **Search Vector Store** → Find closest matching chunks  
- 📑 **Retrieve Chunks** → Get relevant context  
- 🧠 **Generate Answer** → Formulate final response  


In [20]:
user_queries = [
    "What is the Iceberg Index?",
    "What problem does Project Iceberg aim to solve?",
    "How many workers does Project Iceberg simulate?",
    "What are the three main steps of Project Iceberg’s simulation process?",
    "Does the Iceberg Index measure job loss or technical exposure?",
    "What percentage of labor market wage value is represented by current visible AI adoption (Surface Index)?",
    "What percentage is represented by the larger hidden exposure (Iceberg Index)?",
    "How many AI tools were cataloged for the framework?",
    "How many occupations are modeled in the simulation?",
    "Which state has the highest Surface Index according to the paper?",
    "What is meant by 'automation surprise'?",
    "What kinds of tasks does the Surface Index primarily capture?",
    "What kinds of tasks does the Iceberg Index primarily capture?",
    "Does the Iceberg Index predict job displacement? Why or why not?",
    "What is the role of wage-value weighting in the Iceberg Index?",
    "What does the validation against career transition data show?",
    "What is the agreement percentage between the Surface Index and the Anthropic Economic Index?",
    "Which traditional economic metrics fail to capture Iceberg exposure effectively?",
    "Which states show the largest gap between Surface Index and Iceberg Index?",
    "What does the Herfindahl–Hirschman Index (HHI) reveal about exposure concentration?"
]

# Retrieval

In [13]:
def retrieve_context(question, top_k=3):
    q_emb = embed(question)
    sims = embeddings.dot(q_emb) / (np.linalg.norm(embeddings, axis=1) * np.linalg.norm(q_emb))
    sorted_sims_idxes = np.argsort(sims)[::-1]
    idxs = sorted_sims_idxes[:top_k]
    return "\n\n".join(chunks[i] for i in idxs)

# Augmentation

In [14]:
def augment(query, verbose=False):
  context = retrieve_context(query, top_k=3)
  if verbose:
    nbprint('---')
    nbprint(context)
    nbprint('---')

  aug_prompt = f"Context:\n {context} \n \n Question: {query} \n Answer based only on the context."
  return aug_prompt

In [15]:
async def answer_with_agent(prompt):
    res = await rag_agent.run(prompt)
    return res.output

# Usage

In [21]:
query = user_queries[3]
aug_prompt = augment(query, verbose=True)
ans = await answer_with_agent(aug_prompt)
nbprint(f" **Q:** {query} \n\n **A:** {ans} \n\n ---")

---

g systems, provides
the computational execution layer. The infrastructure delivers massively parallel processing across
20

Project Iceberg
thousands of compute nodes with GPU acceleration that supports large-scale optimization and
analysis operations. This computational capacity transforms previously intractable population
modeling challenges into feasible real-time policy analysis tools. These integrated components
provide the computational foundation for the Iceberg Index and its policy scenario testing
functionality, enabling dynamic analysis of workforce transformation scenarios under different
policy conditions.
21



ures,
or regulatory choices—and run scenarios under different assumptions of technology maturity and
adoption. The simulation then yields three kinds of insight: how occupations and skills evolve
over time, where disruption is geographically spread, and how shocks in one sector cascade into
others. These outputs allow policymakers to compare strategies side by side and anticipate both
direct and indirect consequences.
The framework builds on validated foundations (see Appendix A), ensuring that the simulated
dynamics reflect real labor-market structure. Its primary output is the Iceberg Index—a skills-
centered measure of workforce transformation introduced in Section 4.
Project Iceberg Goal: To provide a sandbox for testing workforce strategies before
implementation—enabling policymakers to assess technical exposure, evaluate
interventions under different scenarios, and target preparation where it matters most.
5

Project Iceberg
4
The Iceberg Index
The Iceberg Index is a skills-centered measur

esigning pathways when jobs shift.
Step 2 – AI Workforce Mapping: We catalog over 13,000 AI tools - including copilots
and workflow automation systems - using the same skill taxonomy. This allows direct comparison
between human and AI capabilities. The alignment reveals where AI augments human work
(e.g. automating hospital paperwork so nurses can spend more time with patients) and where it
transforms tasks entirely (e.g. accelerating code generation requires engineers to shift towards
oversight, testing, and integration skills).
Step 3 - Human–AI Simulation: We combine these two populations in Large Population
Models (LPMs), simulating billions of interactions between workers, skills, and AI tools. These
simulations incorporate factors such as technology readiness, adoption behavior, and regional
variation. Policymakers can use the resulting scenarios to anticipate disruption, test reskilling
4

Project Iceberg
Figure 3. The Iceberg Index reveals workforce exposure five times larger than visibl

---

 **Q:** What are the three main steps of Project Iceberg’s simulation process? 

 **A:** The three main steps of Project Iceberg's simulation process are:

1.  **AI Workforce Mapping**: Cataloging over 13,000 AI tools using a skill taxonomy for direct comparison with human capabilities.
2.  **Human–AI Simulation**: Combining workers and AI tools in Large Population Models (LPMs) to simulate interactions, incorporating factors like technology readiness, adoption behavior, and regional variation.
3.  **Scenario Testing**: Using the resulting scenarios to anticipate disruption and test reskilling, or regulatory choices under different assumptions of technology maturity and adoption. 

 ---